# Traffic Tune - Optimizing Traffic Signals with Reinforcement Learning

## Introduction

Welcome to the Traffic Tune POC notebook. Our project focused on optimizing traffic signal control using reinforcement learning. Traffic congestion is a major problem in urban areas, leading to increased travel times, fuel consumption, and pollution. Traditional traffic signal control systems often struggle to adapt to dynamic traffic conditions, resulting in suboptimal traffic flow.

Traffic Tune is a recommendation system that leverages reinforcement learning to dynamically adjust traffic signals at intersections. By learning from traffic patterns in real-time, Traffic Tune aims to improve traffic flow, reduce congestion, and enhance overall transportation efficiency.

In this POC, we will demonstrate how to train a reinforcement learning agent to optimize traffic signal control in a simulated environment. We will use the SUMO (Simulation of Urban MObility) traffic simulation tool and the Stable Baselines3 library to train a Deep Q-Network (DQN) agent to learn an optimal traffic signal control policy.


# Setup and Installations

In [1]:
import env_manager as env_manager
import ppo_trainer as ppo_trainer
import dqn_trainer as dqn_trainer

In [2]:
def chain_training(manager: env_manager, generator: env_manager.EnvManager.env_generator, algo_agent, running_result: list):
    if len(running_result) != 0: 
        # take the best config from the previous training 
        best = running_result[-1].get_best_result("env_runners/episode_reward_max", "max")
        
        # Initialize the environment manager with new route file
        rou, csv = next(generator)
        manager.initialize_env(rou, csv)
        
        # continue the training with the best config
        algo_agent.config = best.config
        algo_agent.build_config()
    
    result = algo_agent.train()
    
    return result

def training(num_intersection: int, experiment_type: str, algo_config: str, env_config: str, num_training_cycles: int):
    running_result = []
    algo_agent = None
    sumo_type = "SingleAgent"
    algo_type = experiment_type.split("_")
    
    if experiment_type.__contains__("Multi"):
        sumo_type = "MultiAgent"
    
    # Initialize the environment manager
    manager = env_manager.EnvManager(f"{sumo_type}Environment", env_config, json_id=f"intersection_{num_intersection}")
    generator = manager.env_generator(f"Nets/intersection_{num_intersection}/route_xml_path_intersection_{num_intersection}.txt", algo_name=algo_type[0])
    
    # Initialize the environment manager with new route file
    rou, csv = next(generator)
    manager.initialize_env(rou, csv)
    
    # Initialize the Algo agent
    if algo_config.startswith("ppo"):
        ppo_agent = ppo_trainer.PPOTrainer(config_path=algo_config, env_manager=manager, experiment_type=experiment_type)
        ppo_agent.build_config()
        algo_agent = ppo_agent
        
    elif algo_config.startswith("dqn"):
        dqn_agent = dqn_trainer.DQNTrainer(config_path=algo_config, env_manager=manager, experiment_type=experiment_type)
        dqn_agent.build_config()
        algo_agent = dqn_agent

    for i in range(num_training_cycles):
        chain_result = chain_training(manager=manager, generator=generator, algo_agent=algo_agent, running_result=running_result)
        if chain_result is not None:
            running_result.append(chain_result)
    
    return running_result

In [5]:
num_intersection_to_train = 1  # Choose which intersection you want to train

# Choose the experiment_type:
# PPO_SingleAgent | PPO_MultiAgent | DQN_SingleAgent | DDQN_SingleAgent | DQN_MultiAgent | DDQN_MultiAgent
experiment_type = "PPO_SingleAgent"  

num_training_cycles = 1

env_config_file_path = "env_config.json"

ppo_config_file_path = "ppo_config.json"

dqn_config_file_path = "dqn_config.json"

In [6]:
results = training(num_intersection=num_intersection_to_train, experiment_type=experiment_type, algo_config=ppo_config_file_path, env_config=env_config_file_path, num_training_cycles=num_training_cycles)

2024-07-24 22:35:08,874	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!
2024-07-24 22:35:08,875	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!
2024-07-24 22:35:08,875	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_learner_workers` has been deprecated. Use `AlgorithmConfig.num_learners` instead. This will raise an error in the future!
2024-07-24 22:35:08,875	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_learner_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_learner` instead. This will raise an error in the future!
2024-07-24 22:35:08,875	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus

(PPO pid=3996) 2024-07-24 22:35:10,794	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!
(PPO pid=3996) 2024-07-24 22:35:10,795	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!
(PPO pid=3996) 2024-07-24 22:35:10,795	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_learner_workers` has been deprecated. Use `AlgorithmConfig.num_learners` instead. This will raise an error in the future!
(PPO pid=3996) 2024-07-24 22:35:10,795	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_learner_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_learner` instead. This will raise an error in the future!
(PPO pid=3996) 2024-07-24 22:35:10,795

(RolloutWorker pid=3998)  Retrying in 1 seconds


(RolloutWorker pid=3998) 2024-07-24 22:35:13,541	INFO policy.py:1272 -- Policy (worker=1) running on CPU.
(RolloutWorker pid=3998) 2024-07-24 22:35:13,541	INFO torch_policy_v2.py:111 -- Found 0 visible cuda devices.


(RolloutWorker pid=3998) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 0 ACT 0 BUF 0)                      


...

Step #2800.00 (2ms ~= 500.00*RT, ~152000.00UPS, TraCI: 32ms, vehicles TOT 3834 ACT 304 BUF35ms, vehicles TOT 2823 ACT 314 BU


(RolloutWorker pid=3998) Warning: Vehicle 'f_1.308' performs emergency braking on lane ':8081633236_1_1' with decel=9.00, wished=4.50, severity=1.00, time=3166.00.


Step #3100.00 (1ms ~= 1000.00*RT, ~314000.00UPS, TraCI: 30ms, vehicles TOT 4253 ACT 314 BU0ms, vehicles TOT 3990 ACT 312 BUF
Step #3600.00 (2ms ~= 500.00*RT, ~154500.00UPS, TraCI: 52ms, vehicles TOT 4944 ACT 309 BUFACT 1 BUF 0)                      
(RolloutWorker pid=4000)  Retrying in 1 seconds
Step #3600.00 (1ms ~= 1000.00*RT, ~338000.00UPS, TraCI: 44ms, vehicles TOT 5048 ACT 338 BU31ms, vehicles TOT 4383 ACT 313 BU


2024-07-24 22:37:30,001	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/eviat/Desktop/Final_Project/Traffic_Tune_Project/Outputs/Training/intersection_1/saved_agent/PPO_2024-07-24_22-35-08' in 0.0068s.
(PPO pid=3996) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/eviat/Desktop/Final_Project/Traffic_Tune_Project/Outputs/Training/intersection_1/saved_agent/PPO_2024-07-24_22-35-08/PPO_PPO_d609e_00000_0_2024-07-24_22-35-08/checkpoint_000002)
2024-07-24 22:37:30,445	INFO tune.py:1041 -- Total run time: 141.60 seconds (141.12 seconds for the tuning loop).


Step #5.00 (0ms ?*RT. ?UPS, TraCI: 656ms, vehicles TOT 9 ACT 9 BUF 1)                     ACT 1 BUF 0)                      


In [ ]:
print(results)